# import necessary libraries

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from textblob import TextBlob
%matplotlib inline 
import plotly as py
import cufflinks as cf
from sklearn.feature_extraction.text import CountVectorizer


In [30]:
from plotly.offline import iplot
py.offline.init_notebook_mode(connected=True)
cf.go_offline()

# load the dataset

In [31]:
reviews_df = pd.read_csv("Etsy_Reviews.csv", names=["reviews", "ratings", "product"])

# Data Cleaning

In [32]:
# Check for missing values
missing_val = reviews_df.isna().sum()

# drop missing values
reviews_df.dropna(inplace=True)

# make a copy
reviews_df_copy = reviews_df.copy()


In [33]:
# Cleaning text data.
review_feature = reviews_df_copy["reviews"]
cleaned_reviews = []
for review in review_feature:
    # clean strings
    remove_char = review.replace("&#39;", "'")
    remove_char_2 = remove_char.replace("&quot;", "'")
    # remove white spaces
    remove_whitespace = " ".join(remove_char_2.split())
    # convert to lowercase
    convert_to_lowercase = remove_whitespace.lower()
    final_text = convert_to_lowercase # this is the final cleaned data.
    cleaned_reviews.append(final_text)

# turning product column text into lowercase
product_feature = reviews_df_copy["product"]
lowercase_product = []
for review in product_feature:
    convert_to_lowercase = review.lower()
    lowercase_product.append(convert_to_lowercase)
# updating the reviews column with the cleaned reviews
reviews_df_copy["reviews"] = cleaned_reviews
reviews_df_copy["product"] = lowercase_product

In [34]:
df = reviews_df_copy.copy() # finalize copy of the dataframe a copy of the cleaned copy

In [35]:
contractions = { 
"ain't": "am not", # / are not / is not / has not / have not
"aren't": "are not", #  / am not
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had", #  / he would
"he'd've": "he would have",
"he'll": "he shall", #  / he will
"he'll've": "he shall have", #  / he will have"
"he's": "he is", # he has / 
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is", # how has / how does
"i'd": "I had", #  / I would
"i'd've": "I would have",
"i'll": "I will", # I shall 
"i'll've": "I will have", # I shall have / 
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had", #  / it would
"it'd've": "it would have",
"it'll": "it will", # it shall / 
"it'll've": "it will have", # it shall have / 
"it's": "it is", # it has / 
"i'm": "i am",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had", #  / she would
"she'd've": "she would have",
"she'll": "she will", # she shall / 
"she'll've": "she will have", # she shall have / 
"she's": "she is", # she has / 
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is", # so as / 
"that'd": "that had", # that would / 
"that'd've": "that would have",
"that's": "that is", # that has / 
"there'd": "there had", #  / there would
"there'd've": "there would have",
"there's": "there is", # there has / 
"they'd": "they had", #  / they would
"they'd've": "they would have",
"they'll": "they will", # they shall / 
"they'll've": "they will have", # they shall have / 
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had", # / we would
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will", # what shall / 
"what'll've": "what will have", # what shall have / 
"what're": "what are",
"what's": "what is", # what has / 
"what've": "what have",
"when's": "when is", # when has / 
"when've": "when have",
"where'd": "where did",
"where's": "where is", # where has / 
"where've": "where have",
"who'll": "who will", # who shall /
"who'll've": "who will have", # who shall have / 
"who's": "who is", # who has / 
"who've": "who have",
"why's": "why is", # why has / 
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had", #  / you would
"you'd've": "you would have",
"you'll": "you will", # you shall /
"you'll've": "you will have", # you shall have / 
"you're": "you are",
"you've": "you have"
}

In [36]:
def cont_to_exp(review):
    if type(review) is str:
        x = review.replace("’", "'")
        y = x.replace("//", "")
        for key in contractions:
            value = contractions[key]
            y = y.replace(key, value)
        return y
    else:
        return y

In [37]:
df["reviews"] = df.reviews.apply(lambda X: cont_to_exp(X))

# Exploratory Data Analysis

In [38]:
# shape
print(f"rows: {df.shape[0]}, columns: {df.shape[1]}")

# columns
print(df.columns)

# information on columns
print(df.info())

rows: 568, columns: 3
Index(['reviews', 'ratings', 'product'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 568 entries, 0 to 898
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  568 non-null    object
 1   ratings  568 non-null    int64 
 2   product  568 non-null    object
dtypes: int64(1), object(2)
memory usage: 17.8+ KB
None


In [39]:
df_copy = df.copy()

## Feature Engineering 
- Creating columns

In [40]:
# get the average word length
def get_avg_word_len(x):
    words = x.split()
    word_len = 0
    for word in words:
        word_len = word_len + len(word)
    
    return word_len/len(words)

In [41]:
df_copy["polarity"] = df_copy.reviews.apply(lambda x: TextBlob(x).sentiment.polarity) # subjective has a biased viewpoint objective is unbiased
df_copy["character_length"] = df_copy.reviews.apply(lambda x: len(x))
df_copy["word_count"] = df_copy.reviews.apply(lambda x: len(x.split()))
df_copy["average_word_length"] = df_copy.reviews.apply(lambda x: get_avg_word_len(x))

# Distribution of sentiment polarity

In [42]:
df_copy.head()

,reviews,ratings,product,polarity,character_length,word_count,average_word_length
0,i love this item! it is so easy to use and so ...,5,wedding planning digital planner inserts templ...,0.561111,51,13,3.000000
1,I have already used this in my notes and my jo...,5,"vintage papers digital stickers pack 200+, vin...",0.375000,224,40,4.625000
2,i bought the realistic bundle by mistake but r...,5,"ipad planner, digital planner, goodnotes plann...",0.111111,138,24,4.791667
3,love this set!!!! seller is very helpful too,5,"glitters digital stickers pack 200+, glitter s...",0.600000,44,8,4.625000
4,lots of very cute stickers in this set!!!,5,"vintage papers digital stickers pack 200+, vin...",1.000000,41,8,4.250000


In [43]:
df_copy["polarity"].iplot(kind="hist", colors="red", bins=50, xTitle = "Polarity", yTitle="Count", title="Sentiment Polarity Distribution")

In [44]:
df_copy["ratings"].iplot(kind="hist", xTitle = "rating", yTitle="count", title="review rating distribution")

# Distribution of review text length and word length

In [45]:
df_copy["character_length"].iplot(kind="hist", xTitle="character length", yTitle="count", title="Character Text Length Distribution")

In [46]:
df_copy["word_count"].iplot(kind="hist", xTitle="word count", yTitle="count", title="Word Count Distribution")

In [47]:
df_copy["average_word_length"].iplot(kind="hist", bins=50, xTitle="Average Word Length", yTitle="count", title="Average Word Length Distribution")

# Distribution of products
- Which products had the most reviews.
- Clean the X values. Turn it into something shorter.

In [48]:
x = df_copy.groupby("product")["reviews"].count()

x.iplot(kind="bar", xTitle="product", yTitle="count", title="Total Reviews Per Product")

In [49]:
def get_top_n_words(x, n):
    vec = CountVectorizer().fit(x)
    bow = vec.transform(x)
    sum_words = bow.sum(axis = 0)
    vec.vocabulary_.items()
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]

def get_top_n_words_bigram(x, n):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(x)
    bow = vec.transform(x)
    sum_words = bow.sum(axis = 0)
    vec.vocabulary_.items()
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]
    
def get_top_n_words_trigram(x, n):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(x)
    bow = vec.transform(x)
    sum_words = bow.sum(axis = 0)
    vec.vocabulary_.items()
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]



In [50]:
df_copy

,reviews,ratings,product,polarity,character_length,word_count,average_word_length
0,i love this item! it is so easy to use and so ...,5,wedding planning digital planner inserts templ...,0.561111,51,13,3.000000
1,I have already used this in my notes and my jo...,5,"vintage papers digital stickers pack 200+, vin...",0.375000,224,40,4.625000
2,i bought the realistic bundle by mistake but r...,5,"ipad planner, digital planner, goodnotes plann...",0.111111,138,24,4.791667
3,love this set!!!! seller is very helpful too,5,"glitters digital stickers pack 200+, glitter s...",0.600000,44,8,4.625000
4,lots of very cute stickers in this set!!!,5,"vintage papers digital stickers pack 200+, vin...",1.000000,41,8,4.250000
...,...,...,...,...,...,...,...
894,rina is amazing! she answered all of my questi...,5,"ipad planner, digital planner, goodnotes plann...",0.666667,209,38,4.526316
895,absolutely love this sticker set,5,1000+ digital stickers pack - pre-cropped good...,0.500000,32,5,5.600000
896,amazing planner! everything that i need all in...,5,teacher digital planner 2021 2022 with 1000+ s...,0.416667,114,17,5.764706
897,highly detailed organizer. very very difficult...,4,homeschool digital planner 2021 2022 digital p...,0.005195,186,36,4.194444


In [51]:
words = get_top_n_words(df["reviews"], 20)

In [52]:
df1 = pd.DataFrame(words, columns=["Unigram", "Frequency"])
df1 = df1.set_index("Unigram")

In [53]:
df1.iplot(kind="bar", xTitle="Unigram", yTitle="count", title="Top 20 words")

In [54]:
bigram_words = get_top_n_words_bigram(df["reviews"], 20)
df2 = pd.DataFrame(bigram_words, columns=["Bigram", "Frequency"])
df2 = df2.set_index("Bigram")
df2.iplot(kind="bar", xTitle="Bigram", yTitle="count", title="Top 20 words")

In [55]:
trigram_words = get_top_n_words_trigram(df["reviews"], 20)
df3 = pd.DataFrame(trigram_words, columns=["trigram", "Frequency"])
df3 = df3.set_index("trigram")
df3.iplot(kind="bar", xTitle="trigram", yTitle="count", title="Top 20 words")

In [56]:
store_reviews_in_list = []
for review in df_copy["reviews"]:
    store_reviews_in_list.append(review)

combined_reviews = " ".join(store_reviews_in_list)

In [57]:
from wordcloud import wordCloud, STOPWORDS

ImportError: cannot import name 'wordCloud' from 'wordcloud' (C:\Users\Kn133\Anaconda3\envs\etsy-scraper\lib\site-packages\wordcloud\__init__.py)